# Basic image formation

In [ ]:
import holoviews as hv
import holoviews.operation.datashader as hd
hv.extension('bokeh')

## Phase history simulation
The goal of this is to simulate what the received demodulated signal for a SAR system is. Recall from the previous notebook that the zero IF signal we sample is given by:
$$x_{IF}(t, \tau) = Ae^{\frac{-4j\pi R(\tau)}{\lambda}}e^{j\pi \gamma (t-\frac{2R(\tau)}{c})^2}$$
To take digitization into consideration then we will convert continuous time into discrete time by sampling the signal which means we can re-write this as:

$$x_{IF}(n, m) = Ae^{\frac{-4j\pi R[m]}{\lambda}}e^{j\pi \gamma (n-f_s\frac{2R[m]}{c})^2}$$
We'll use the following definition of our system for this simulation
## System parameters

In [ ]:
import numpy as np
from scipy.constants import c as speed_of_light
# Geometry 
arp_poly=np.array([
    [4800897.883632624, 57.036013951381584], [2752862.8001471036, -4.058634127391252], [3180199.1684093135, -82.03914066808467]
])
p_scp=np.array([4787610.688267582, 2764128.319646417,  3170373.7353836414])

num_pulses = 2000 # number of pulses we transmit
prf = 4000 # pulse repition frequency in Hz
bandwidth = 100e6 # trasnmit RF bandwidth in Hz
tx_pulse_len = 1e-05 # Duration of the pulse we send in seconds
t_start = 0.0 # time when we start transmitting pulses in seconds
t_end = 0.4998734257422694 # time of last pulse we sent in seconds
chirp_rate = bandwidth / tx_pulse_len # chirp rate in Hz/s
fc = 10e9 # center frequency of the system in Hz
wavelen = speed_of_light / fc # wavelenght of the center frequecny in meters
rx_window_factor = 3 # multiplier on the pulse length for how long we listen
rx_nyquist_factor = 1.25 # how much over nyquist rate we oversample
sampling_rate = 2 * bandwidth * rx_nyquist_factor # ADC sampling rate in Hz 
sampling_period = 1 / sampling_rate # ADC sampling period in seconds
rx_window_duration = rx_window_factor * tx_pulse_len # Duration of the receive window in seconds
tx_times = np.arange(0, t_end, 1/prf) # Array of times we send pulses in seconds
t_adc = np.arange(0, rx_window_duration, sampling_period) # ADC sampling times in seconds
t_adc -= t_adc.mean()
num_samples = t_adc.size

## Scene definition
To define a scene (scatterers) to image we'll define them in the slant plane coordinate system

In [ ]:
import numpy.polynomial.polynomial as npp
def slant_plane_to_ecef(p_arp_ecef, v_arp_ecef, p_grp_ecef):
    x = p_arp_ecef - p_grp_ecef
    slt_rg = np.linalg.norm(x, axis=-1)
    u_x = x / slt_rg
    z = np.cross(u_x, v_arp_ecef, axisa=-1, axisb=-1)

    look = 1
    u_z = np.expand_dims(look, axis=-1) * (z / (np.linalg.norm(z, axis=-1)))
    u_y = np.cross(u_z, u_x, axisa=-1, axisb=-1)
    slant_to_ecef = np.stack((u_x, u_y, u_z), axis=-1)
    return slant_to_ecef

coa_time = (t_end - t_start) / 2 
p_coa = npp.polyval(coa_time, arp_poly.T).T
v_coa = npp.polyval(coa_time, npp.polyder(arp_poly.T)).T
sp_2_ecef = slant_plane_to_ecef(p_coa, v_coa, p_scp)

In [ ]:
import matplotlib.pyplot as plt
# points_sp = np.stack([
#     np.array([-1000, -500, 0]),
#     np.array([-400, 400, 0]),
#     np.array([0, 0, 0]),
#     np.array([300, 300, 0]),
#     np.array([600, -600, 0]),
# ])
points_sp = np.stack([
    np.array([400, 0, 0]),
])
plt.figure()
plt.plot(points_sp[:, 1], points_sp[:, 0], 'o')
plt.xlabel('Slant plane y (meters) (azimuth direction)')
plt.ylabel('Slant plane x (meters) (range direction)')
plt.title('Slant plane point locations')
plt.grid()
plt.show()

Next we'll convert the slant plane positions to ECEF so we can use them in the simulation

In [ ]:
rotation = sp_2_ecef @ points_sp.T
points_ecef = rotation.T + p_scp

## Simulation
Now we'll simulate the signal the radar receives over time

In [ ]:
import numpy as np
import numpy.polynomial.polynomial as npp
from scipy.constants import c as speed_of_light
targets = [points_ecef[ii,:].flatten() for ii in range(points_ecef.shape[0])]
phase_history_per_target = np.zeros((len(targets), num_pulses, num_samples), dtype=np.complex128)

def lfm(samp_time):
    phase = np.pi * chirp_rate * samp_time * samp_time
    half_dur = 0.5 * tx_pulse_len
    t_max, t_min = half_dur, -half_dur

    mag = np.logical_and(
        np.less_equal(samp_time, t_max),
        np.greater_equal(samp_time, t_min)
    ).astype(float)

    return mag * (np.cos(phase) + 1j * np.sin(phase))

for i_tgt, tgt_pos in enumerate(targets):
    for i_pulse in range(num_pulses):
        # Evaluate ARP/GRP poly
        arp_pos = npp.polyval(tx_times[i_pulse], arp_poly.T).T
        # TODO: Use GRP poly for scan
        grp_pos = p_scp

        # Compute distance from the ARP to SCP
        grp_distance = np.linalg.norm(grp_pos - arp_pos)

        # grp_time_delay is the time delay from the ARP to the GRP
        grp_time_delay = 2 * grp_distance / speed_of_light
        
        # compute distance (time delay) from the arp to the target
        monostatic_distance = 2 * np.linalg.norm(tgt_pos - arp_pos)

        # Using the monostatic approximation we will receive the pulse rx_time
        # past tx_time
        rx_time = monostatic_distance / speed_of_light

        # Compute slow time term
        slow_time_term = np.exp(-4.0j * np.pi * monostatic_distance / wavelen)

        # Here we want the receive times relative to the delay to the scene center point
        # We want to do this so that after motion compensation a target at the mocomp point will show up 
        # at the center of the receive window.  Here we choose the mocomp point to be the scp (grp)
        t_delay = rx_time - grp_time_delay

        # Center the ADC around the time delay to the target.  We will clip this to the receive window
        # in the lfm function. i,e) if a time delay is outside of the receive window the response will
        # be all 0s
        t_fast = t_adc - t_delay

        # Compute fast time term.  
        fast_time_term = lfm(t_fast)

        # couple slow time term and fast time term
        phase_history_per_target[i_tgt, i_pulse, :] = slow_time_term * fast_time_term 


## Visualization
Let's look at what the phase history looks like for each target

In [ ]:
phase_histories = [
    hd.rasterize(
        hv.Image(
            np.real(phase_history_per_target[ii, :, :]),
            bounds=(0, 0, num_samples, num_pulses),
        ).opts(
            width=400,
            height=400,
            cmap='gray',
            title=f'Target Slant Plane Location {points_sp[ii, 0]}, {points_sp[ii, 1]}',
            xlabel='Fast time sample',
            ylabel='Slow time sample',
        )
    )
    for ii in range(len(targets))
]
layout = hv.Layout(phase_histories).cols(2 if len(targets) >= 2 else 1)
layout

In [ ]:
phase_histories = [
    hd.rasterize(
        hv.Image(
            np.unwrap(np.angle(phase_history_per_target[ii, :, :])),
            bounds=(0, 0, num_samples, num_pulses),
        ).opts(
            width=400,
            height=400,
            # cmap='gray',
            title=f'Target Slant Plane Location {points_sp[ii, 0]}, {points_sp[ii, 1]}',
            xlabel='Fast time sample',
            ylabel='Slow time sample',
            colorbar=True,
        )
    )
    for ii in range(len(targets))
]
layout = hv.Layout(phase_histories).cols(2 if len(targets) >= 2 else 1)
layout

## Pulse returns in the receive window
Let's examine what the return for a single pulse for each target looks like in the receive window

In [ ]:
target_returns = [
    hv.Curve(
        np.abs(phase_history_per_target[ii, 0, :]),
        label=f'Target sp_x location {points_sp[ii, 0]}'
    ).opts(
        xlabel='fast time index',
        title='Receive Window for single pulse',
        show_grid=True,
    ) for ii in range(len(targets))
]
hv.Overlay(target_returns).opts(width=600, height=600)

The signal we sample is actually the superposition of all scatterers in the scene.  This means we dont get a phase history per point but rather the phase history of the combination of **all** points.  This means we need to add all the phase histories together

In [ ]:
phase_history = phase_history_per_target.sum(axis=0)
print(phase_history.shape)

In [ ]:
hd.rasterize(
    hv.Image(
        np.real(phase_history),
        bounds=(0, 0, num_samples, num_pulses)
    ).opts(
        width=600,
        height=600,
        cmap='gray'
    )
)

# Basic image formation
Our goal is to take the data and form an estimate of the imaged scene.  This estimate is the SAR image.  The first two steps we'll do is 
1. Resolve range
2. Compensate for our relative motion

## Image formation step one: Range matched filter
We covered matched filtering already.  Here we construct the matched filter based off of the waveform function $h(t)$.  This step is so for each pulse we know distance to our objects.  

The goal of this is to determine range to all points in the scene 

In [ ]:
ref_waveform = lfm(t_adc)
matched_filter = np.conj(ref_waveform[::-1])
matched_filter_freq = np.fft.fftshift(np.fft.fft(matched_filter))
range_dispersed = np.fft.fftshift(np.fft.fft(phase_history, axis=1), axes=1)
range_matched_filtered = range_dispersed * matched_filter_freq

Instead of convolution here we matched filtered by multiplying in the frequency domain. We can take a FFT of this data to take it back to the compressed domain.  We should expect to see range resolved which means we will see a spike in range for each target which spans all of azimuth because we have not figured out point locations in azimuth yet 

In [ ]:
range_compmressed = np.fft.ifftshift(np.fft.ifft(range_matched_filtered, axis=1), axes=1)

In [ ]:
hd.rasterize(
    hv.Image(
        np.log10(
            np.abs(range_compmressed)
        ),
        bounds=(0, 0, num_samples, num_pulses),
    ).opts(
        width=800,
        height=800,
        cmap='gray',
        xlabel='Range (Near Range <--> Far Range)',
        ylabel='Pulses',
        title='Range compressed'
    )
)

## Step two: Azimuth Matched filter (motion compensation)

Motion compensation is the undoing of unwanted signal phase induced on the image focal point, known as the Scene Center Point (SCP). In the 
compressed domain, the SCP signal return should compress to a sinc-like function at DC (zero frequency) so that the center imaging point displays at the center of the image.

To compress the SCP to a single sinc-like function at the image center in the compressed domain, the SCP signal return must be a constant-phase rect 
function in the frequency domain.  

Motion compensation has two components:
* Along slow-time, the motion of the transmitter and receiver platforms induce a phase modulation.
* Along fast-time there will be induced linear phase if the receiver sampling window does not center on the SCP return pulse.
    - I do not model this here

Overall, the goal of mocomp is to remove the fast-time and slow-time modulation induced upon the SCP return. 

### Derivation
Recall our signal model for the sampled signal is 
$$x_{IF}(n, m) = Ae^{\frac{-4j\pi R[m]}{\lambda}}e^{j\pi \gamma (n-n_{fast}[m])^2}$$
Let $e^{j\pi \gamma (n-n_{fast}[m])^2} = h(n - n_{fast}[m])$ which means we can rewrite this as
$$x_{IF}(n, m) = Ah(n - n_{fast}[m])e^{\frac{-4j\pi R[m]}{\lambda}}$$

Lastly, we will re-write the aximuth term back in terms of time delays instead of range:
$$x_{IF}(n, m) = Ah(n - n_{fast}[m])e^{-j\omega \tau[m]}$$

The signal sampled along slow time is in the fourier domain already, so to convert fast time into the frequency domain we'll take the fourier transform
$$\mathcal{F}(x_{IF}(n, m)) = H(\omega)e^{-j\omega [n]n_{fast}[m]}e^{-j\omega \tau[m]}$$
Note that $\omega[n]$ is a function of the fast time sample $n$.  

The goal of motion comensation is to remove these unwanted phase terms to recover exactly $H(\omega)$ for the scene center point  

### Scene center point model

Let's assume that we are imaging a single point positioned at the scene center point.  The pulse will be transmitted at time $t_{tx}$, and be received a time later at $t_{rx}$.  The receiver window will then center on $t_{rx}$.  This means that the sample that the SCP return shows up will be at
$$n_{scp} = f_s (\tau_{scp} - t_{rx})$$
Where $\tau_{scp}$ is the two way time delay from the transmitter to the SCP back to the receiver.  

This means for a single point scatterer at the SCP the dispersed (frequency domain) SAR signal can be written as:
$$X_{IF}(n, m) = H(\omega)e^{-j\omega [n]n_{scp}[m]}e^{-j\omega \tau_{scp}[m]}$$  

We want to remove these unwanted phase terms for the SCP so to do that we will form two corrections; one for Doppler and one for Range.  Note: to undo these terms the corrections will just be the complex conjugate of the unwanted terms  

**Doppler correction**:  
This term will undo the slow time modulation which changes from pulse to pulse due to our relative motion between the ARP and the SCP.  This makes it so that the relative doppler frequecny for the SCP will be 0 Hz causing it to show up at the center of the image in azimuth 
$$C_{d} = e^{j\omega \tau_{scp}[m]}$$
**Range Correction**:  
The linear phase term across phase time will cause a sample shift in the compressed domain.  We want the SCP range to show up at DC in the image, so to do that we will undo this linear phase ramp.  This compensates for in general when your receive window is not perfectly center on your scene center point.  
$$C_{r} = e^{j\omega [n]n_{scp}[m]}$$

**REMEMBER I DID NOT MODEL THIS IN THIS NOTEBOOK SO I WILL OMIT THIS CORRECTION**

Applying these two corrections we see the dispersed signal after mocomp for a single point scatterer at the SCP is given by:
$$X_{IF}(n, m) = (H(\omega)e^{-j\omega [n]n_{scp}[m]}e^{-j\omega \tau_{scp}[m]})(e^{j\omega \tau_{scp}[m]})( e^{j\omega [n]n_{scp}[m]})$$
$$X_{IF}(n, m) = H(\omega)$$
For the SCP point now all slow time modulations are removed.  

### Post mocomp for point scatterer not at scene center point

$$X_{IF}(n, m) = H(\omega)e^{-j\omega [n](n_{fast}[m] - n_{scp}[m])}e^{-j\omega(\tau[m] -  \tau_{scp}[m])}$$

This will make the signal we sample contain relative doppler frequency to the SCP for each point in the scene  

# Visualizing what mocomp does
Let's examine what this looks like for the points we simulated


In [ ]:
arp_pos = npp.polyval(tx_times, arp_poly.T).T
scp_slant_range = np.linalg.norm(grp_pos - arp_pos, axis=-1)

We will effectively be referencing all of our distances to each point over slow time to the distance to the SCP over slow time.  Let's compute that compensated distance

In [ ]:
compensated_distance = []
for tgt_pos in targets:
    distance_to_target = np.linalg.norm(tgt_pos - arp_pos, axis=-1)
    compensated_distance.append(distance_to_target - scp_slant_range)

Let's see what the compensated distance looks like for each point scatterer

In [ ]:
distances = [
    hv.Curve(
        distance,
    ).opts(
        width=400,
        height=400,
        title=f'Target Slant Plane Location {points_sp[ii, 0]}, {points_sp[ii, 1]}',
        show_grid=True,
        xlabel='Pulse Index',
        ylabel='Compensated distance (m)'
    )
    for ii, distance in enumerate(compensated_distance)
]
layout = hv.Layout(distances).cols(2 if len(targets) >= 2 else 1).opts(shared_axes=False)
layout

There are a few interesting observations (recall that the distance to a point scatterer is embedded as a phase term in our singal model)
* The comesated distance for the SCP is 0 over slow time
* For each point that is offset in azimuth there is a linear relative distance to the SCP over slow time
    - This means each point target that is offset in azimuth will have a linear phase term
 
Recall that the derivative of phase is frequency.  This means that after mocomp the linear phase ramp corresponse to an azimuth frequency that uniquely encodes the azimuth position of a point relative to the SCP!  This is how we determine spatially where a point is in azimuth after resolving range

## Performing motion compensation

In [ ]:
scp_delay = 2 * scp_slant_range / speed_of_light
correction = 2 * np.pi * fc * scp_delay
cphd = range_matched_filtered * np.exp(1j * correction[:, np.newaxis])

## Step three: form image
At this point we have determined range and a point's spatial location meaning that we have formed an estimate of the fourier transform of the reflectivity function of our scene.  To form the image we can take a FFT of the spectrum

In [ ]:
img = np.fft.ifftshift(np.fft.ifft2(cphd), axes=(0, 1))
img_with_noise = img + 1/np.sqrt(2) * (np.random.randn(num_pulses, num_samples) + 1j*np.random.randn(num_pulses, num_samples))

In [ ]:
img_db = np.log10(np.abs(img))
lims = np.percentile(img_db.flatten(), q=[90, 99.9]) 
hd.rasterize(
    hv.Image(img_db)
).opts(
    width=800,
    height=800,
    cmap='gray',
    clim=(np.min(lims), np.max(lims)),
    xlabel='Range (Near Range <--> Far Range)',
    ylabel='Azimuth',
)

We see that we have a 2D sinc function corresponding to the imaged points spatial location!  

We negletected system noise terms, so let's see what the image looks like with noise added

In [ ]:
img_db_with_noise = np.log10(np.abs(img_with_noise + 1e-16))
from sarpy.visualization.remap import pedf
hd.rasterize(
    hv.Image(20*np.log10(img_db_with_noise))
).opts(
    width=800,
    height=800,
    cmap='gray',
    xlabel='Range (Near Range <--> Far Range)',
    ylabel='Azimuth',
)


# Is that all we need to do to make an image?
No! If we did a perfect job we would expect to see the phase of our image to be a 2D sinusoid where the range frequency describes the range position of each scatterer and the azimuth frequency describing the azimuth poisition of each scatterer.  

Let's see what the sinusoid looks like for our data:

In [ ]:
# img_db = np.log10(np.abs(img))
# lims = np.percentile(img_db.flatten(), q=[50, 99.9]) 
hd.rasterize(
    hv.Image(np.real(cphd))
).opts(
    width=800,
    height=800,
    cmap='gray',
    # clim=(np.min(lims), np.max(lims)),
    xlabel='Range',
    ylabel='Azimuth',
)

Notice how the data seems to curved over azimuth instead of being constant.  This is because our data collection acutally lives on a polar grid and we have not accounted for that yet.  The act of correcting the phase such that it goes from being on a curved arc to be rectangular is known as polar formatting!  We will cover this in the next notebook!

## Why did my image look good?
* Dwell was short so there was not enough anglular disparity on the collection which means the polar grid was effectively rectilinear already
* Point targets were constrained near the scene center point
    - The farther out from the mocomp point you go the more variation you will see in the polar grid and the data will be less focused

## What does it normally look like if you don't polar format?
It turns out there is a product you can order for SAR data which has been matched filtered and motion compensated.  That product is known as a compensated phase history data (CPHD).  Let's see what taking a 2D FFT of a CPHD from umbra looks like

In [ ]:
from sarpy.io.phase_history.cphd import CPHDReader
cphd_path = '/mnt/c/Users/Austin/Documents/GitHub/radar_learning/7_basic_image_formation/2024-01-12-04-09-18_UMBRA-05_CPHD.cphd'
reader = CPHDReader(cphd_path)

In [ ]:
ph = reader[:, :]

In [ ]:
import numpy as np
cphd_img = np.fft.fftshift(np.fft.fft2(ph), axes=(0, 1))

In [ ]:
from sarpy.visualization.remap import pedf
hd.rasterize(
    hv.Image(pedf(cphd_img))
).opts(
    width=800,
    height=800,
    cmap='gray',
    # clim=(np.min(lims), np.max(lims)),
    xlabel='Range',
    ylabel='Azimuth',
)

Notice how the data looks focused near the center of the image but gets more and more blurry as you traverse farthere away from the center.  THis is because the data has not been polar formatted yet.  I won't spoil this too much but you can't take a **fast** fourier transform on data that is irregularly sampled.  Before polar formatting the data is irregularly sampled in azimuth.  The act of polar formatting is making it so that data lives on a support with even sampling so you can make use of the **fast** fourier transform

## Impulse response I NEVER FINISHED THIS

In [ ]:
peak = np.unravel_index(
    np.argmax(img_db),
    img_db.shape
)
print(peak)
print(img_db.shape)
mid_pulse, mid_sample = peak


In [ ]:
rg_ipr = img_db[mid_pulse, :].flatten()
plt.figure()
plt.plot(20*rg_ipr)


In [ ]:
dp_ipr = img_db[:, mid_sample].flatten()
plt.figure()
plt.plot(20*dp_ipr)